# Instrument Classification - Part 2: Model Architecture & Training

In [ ]:
import numpy as np, keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,Activation,BatchNormalization,LeakyReLU

instr = ['pia','gac','gel','vio','cel','sax','tru','cla','voi']
mix   = ['pia-gac','pia-gel','pia-vio','pia-cel','pia-sax','pia-tru','pia-cla','pia-voi',
         'gac-gel','gac-vio','gac-cel','gac-sax','gac-tru','gac-cla','gac-voi','gel-vio',
         'gel-cel','gel-sax','gel-tru','gel-cla','gel-voi','vio-cel','vio-sax','vio-tru',
         'vio-cla','vio-voi','cel-sax','cel-tru','cel-cla','cel-voi','sax-tru','sax-cla',
         'sax-voi','tru-cla','tru-voi','cla-voi']
#mix = []

data_path = '/gpu-data2/chgar/IC/CQT_npys/'

In [ ]:
def to_categorical(list):
    list_final = []
    for label in list: list_final.append(label.split('-'))
    temp_list = [[0]*len(instr)]*(len(list_final))
    for i in range(len(list_final)):
        x = [0]*len(instr)
        x.insert(0,1)
        x.pop()
        for ins in instr:
            if (ins in list_final[i]): temp_list[i] = [sum(x) for x in zip(temp_list[i],x)]
            x.insert(0,x.pop())
    return temp_list

def mixed(solo_num,mix_num):
    y_train,y_test = [],[]
    first = True
    for ins in (instr+mix):
        print("Loading:",ins)
        x_train = np.load(data_path+ins+'/x_train_' + ins + '.npy')
        x_test  = np.load(data_path+ins+'/x_test_' + ins + '.npy')
        if ins in instr: limit = solo_num
        else: limit = mix_num

        for i in range(int(0.9*limit)): y_train.append(ins)
        for i in range(int(0.1*limit)): y_test.append(ins)
        if first:
            first = False
            x_train_final = x_train[0:int(0.9*limit)]
            x_test_final = x_test[0:int(0.1*limit)]
        else:
            x_train_final = np.concatenate([x_train_final,x_train[0:int(0.9*limit)]],axis=0)
            x_test_final = np.concatenate([x_test_final, x_test[0:int(0.1*limit)]],axis=0)

    y_train = np.array(to_categorical(y_train))
    y_test = np.array(to_categorical(y_test))
    id1 = np.random.permutation(len(x_train_final))
    id2 = np.random.permutation(len(x_test_final))
    x_train,y_train = x_train_final[id1], y_train[id1]
    x_test,y_test = x_test_final[id2], y_test[id2]

    return (x_train,x_test,y_train,y_test)

In [ ]:
solo_num = 5000 #between 0 and 5000
mix_num  = 5000 #between 0 and 5000
(x_train,x_test,y_train,y_test) = mixed(solo_num,mix_num)
print("Train Size:", x_train.shape)
print("Test  Size:", x_test.shape)

In [ ]:
model = Sequential()

model.add(Conv2D(64,kernel_size=(3,3),padding='same',strides=1,input_shape = x_train[0].shape))
#model.add(Conv2D(64,kernel_size=(3,3),padding='same',strides=1))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.3))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128,kernel_size=(3,3),padding='same',strides=1))
#model.add(Conv2D(128,kernel_size=(3,3),padding='same',strides=1))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.3))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256,kernel_size=(3,3),padding='same',strides=1))
#model.add(Conv2D(256,kernel_size=(3,3),padding='same',strides=1))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.3))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.2))

model.add(Conv2D(640,kernel_size=(3,3),padding='same',strides=1))
#model.add(Conv2D(640,kernel_size=(3,3),padding='same',strides=1))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.3))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(1024))
model.add(LeakyReLU(alpha=0.3))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(9))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(lr=0.0001),metrics=['accuracy'])
history = model.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_test,y_test))
model.save('../Models/ICmodel_....h5')